In [1]:
import os
import numpy as np
from sklearn.decomposition import PCA
from LUT import *
import torch
import pdb
from losses import psnr_from_mse
import matplotlib.pyplot as plt
from visualize import *
from numpy.linalg import matrix_rank
from scipy.linalg import orth

In [2]:
# name = "/mnt/notvmn/fiveK/Test+20+17_models/model0200"
# name = "/mnt/notvmn/bit16_expertC_Minus1.5/Test+20_models/model0243"
# name = "/mnt/notvmn/input/Test+20+33+pq_models/model0293"
# name = "Test+20_fiveK_models_model0383_LUTs0"
# name = "model0293 0293_0.0000--25.4170_0.7721--0.8843_best"
# key = 'LUT_model.LUTs'
# model = torch.load(name + ".pth", map_location=torch.device('cpu')) #没法用GPU加速
# luts = model[key]


root = "/mnt/tvmn1/input"
name = "Test+20-1-1_models/"
epoch = 181
name = "Test+10-1-1_models/"
epoch = 335

# root = "/mnt/notvmn/input"
# name = "Test+20-1-1_models/"
# epoch = 280

model = torch.load(os.path.join(root, name, "model{:0>4}.pth".format(epoch)),map_location=torch.device('cpu'))
cubes = model['LUT_model.LUTs'].reshape(-1,3,33,33,33)
luts = cube_to_lut(cubes)


In [3]:
model.keys()

odict_keys(['backbone.model.0.weight', 'backbone.model.0.bias', 'backbone.model.2.weight', 'backbone.model.2.bias', 'backbone.model.3.weight', 'backbone.model.3.bias', 'backbone.model.5.weight', 'backbone.model.5.bias', 'backbone.model.6.weight', 'backbone.model.6.bias', 'backbone.model.8.weight', 'backbone.model.8.bias', 'backbone.model.9.weight', 'backbone.model.9.bias', 'backbone.model.11.weight', 'backbone.model.11.bias', 'backbone.model.12.weight', 'backbone.model.12.bias', 'classifier.0.weight', 'classifier.0.bias', 'classifier.3.weight', 'classifier.3.bias', 'LUT_model.LUTs'])

In [6]:
luts.shape

torch.Size([10, 3, 33, 33, 33])

In [7]:
dim=33
n = luts.shape[0]

In [8]:
# if luts.shape[1] == 17**3*3 or luts.shape[1] == 17*3:
#     dim=17
# elif luts.shape[1] == 33**3*3 or luts.shape[1] == 33*3:
#     dim = 33
# elif luts.shape[1] == 64**3*3 or luts.shape[1] == 64*3:
#     dim = 64
# luts = (luts*dim).int()
# luts = luts.reshape(n,3,dim,dim,dim).cpu()
# print(luts.shape, luts.min(), luts.max())
# cubes = lut_to_cube(luts)
# cubes = luts
p_data = cubes.permute(0,1,3,4,2).reshape(n*3*dim*dim,-1)
print(p_data.shape)
print(matrix_rank(p_data))

torch.Size([32670, 33])
32


In [9]:
layer_data = cubes.reshape(n*3*dim,-1)
print(layer_data.shape)
print(matrix_rank(layer_data))
# orth_data = orth(layer_data)
# print(orth_data.shape)

torch.Size([990, 1089])
804


In [7]:
# q
c_ls = [(9,9),(17,17),(25,25)]
for condition in c_ls:
    target = cubes
    down = torch.nn.Upsample(condition)
    up = torch.nn.Upsample((dim,dim))
    down_data = down(target.reshape(n,3*dim,dim,dim))
    print(down_data.shape)
    rec_data = up(down_data)
    rec_data = rec_data.reshape(n,3,dim,dim,dim)
#     np.save("new_data q "+str(condition), rec_data)
    print(psnr_from_mse(np.power((target-rec_data),2).mean(), 1).item())

torch.Size([20, 99, 9, 9])
39.14540100097656
torch.Size([20, 99, 17, 17])
42.04197692871094
torch.Size([20, 99, 25, 25])
43.10335159301758


In [6]:
# p
c_ls = [9,17,25]
for condition in c_ls:
    down = torch.nn.Upsample(condition)
    up = torch.nn.Upsample(dim)
    down_data = down(p_data.reshape(n,3*dim*dim,dim))
    print(down_data.shape)
    rec_data = up(down_data)
    rec_data = rec_data.reshape(n,3,dim,dim,dim).permute(0,1,4,2,3)
    np.save("new_data p "+str(condition), rec_data)
    print(psnr_from_mse(np.power((cubes-rec_data),2).mean(), 1).item())

torch.Size([20, 3267, 9])
25.465147018432617
torch.Size([20, 3267, 17])
26.53583526611328
torch.Size([20, 3267, 25])
27.359941482543945


In [24]:
# all
c_ls = [(9,9,9),(17,17,17),(25,25,25)]
for condition in c_ls:
    target = cubes
    down = torch.nn.Upsample(condition)
    up = torch.nn.Upsample((dim,dim,dim))
    down_data = down(target)
#     print(down_data.shape)
    rec_data = up(down_data)
    # rec_data = down_data
    print(rec_data.shape)
    np.save("new_data all "+str(condition), rec_data)
    print(psnr_from_mse(np.power((target-rec_data),2).mean(), 1).item())

torch.Size([20, 3, 33, 33, 33])
24.447603225708008
torch.Size([20, 3, 33, 33, 33])
26.549331665039062
torch.Size([20, 3, 33, 33, 33])
27.503501892089844


In [8]:
#q
c_ls = [0.9, 0.95, 0.99]
for condition in c_ls:
    pca = PCA(condition)
    pca.fit(layer_data)
    print(pca.components_.shape)
    np.save("bases/bases of q " + str(condition), pca.components_)
    new_data = pca.transform(layer_data) # 104*3*d, s
    rec_data = np.matmul(new_data, pca.components_) # 104*3*d, d*d
    print(new_data.shape, rec_data.shape)
    rec_data = torch.from_numpy(rec_data.reshape(n,3,dim,dim,dim)).type(torch.float32)
    np.save("new_data/new_data q "+str(condition), rec_data)
    print(psnr_from_mse(np.power((cubes-rec_data),2).mean(), 1).item())

(2, 1089)
(990, 2) (990, 1089)
34.106910705566406
(4, 1089)
(990, 4) (990, 1089)
35.24885559082031
(20, 1089)
(990, 20) (990, 1089)
36.73173141479492


In [11]:
# q
c_ls = [10,20,200]
for condition in c_ls:
    pca = PCA(condition)
    pca.fit(layer_data)
    print(pca.components_.shape)
    np.save("10_bases/bases of q " + str(condition), pca.components_)
    new_data = pca.transform(layer_data) # 104*3*d, s
    rec_data = np.matmul(new_data, pca.components_) # 104*3*d, d*d
    print(new_data.shape, rec_data.shape)
    rec_data = torch.from_numpy(rec_data.reshape(n,3,dim,dim,dim)).type(torch.float32)
    np.save("10_new_data/new_data q "+str(condition), rec_data)
    print(psnr_from_mse(np.power((cubes-rec_data),2).mean(), 1).item())

(10, 1089)
(990, 10) (990, 1089)
36.57631301879883
(20, 1089)
(990, 20) (990, 1089)
37.035911560058594
(200, 1089)
(990, 200) (990, 1089)
37.5155029296875


In [9]:
# p
c_ls = [0.9, 0.95, 0.99]
for condition in c_ls:
    pca = PCA(condition)
    pca.fit(p_data)
    print(pca.components_.shape)
    np.save("bases of p " + str(condition), pca.components_)
    new_data = pca.transform(p_data) # 104*3*d, s
    rec_data = np.matmul(new_data, pca.components_) # 104*3*d, d*d
    print(new_data.shape, rec_data.shape)
    rec_data = torch.from_numpy(rec_data.reshape(n,3,dim,dim,dim).transpose(0,1,4,2,3)).type(torch.float32)
    np.save("new_data p "+str(condition), rec_data)
    print(psnr_from_mse(np.power((cubes-rec_data),2).mean(), 1).item())

(3, 33)
(65340, 3) (65340, 33)
39.38983154296875
(4, 33)
(65340, 4) (65340, 33)
40.65502166748047
(8, 33)
(65340, 8) (65340, 33)
42.06874084472656


In [11]:
# p
c_ls = [3,5,7,9]
for condition in c_ls:
    pca = PCA(condition)
    pca.fit(p_data)
    print(pca.components_.shape)
    np.save("bases of p " + str(condition), pca.components_)
    new_data = pca.transform(p_data) # 104*3*d, s
    rec_data = np.matmul(new_data, pca.components_) # 104*3*d, d*d
    print(new_data.shape, rec_data.shape)
    rec_data = torch.from_numpy(rec_data.reshape(n,3,dim,dim,dim).transpose(0,1,4,2,3)).type(torch.float32)
    np.save("new_data p pca"+str(condition), rec_data)
    print(psnr_from_mse(np.power((cubes-rec_data),2).mean(), 1).item())

(3, 33)
(65340, 3) (65340, 33)
27.568796157836914
(5, 33)
(65340, 5) (65340, 33)
29.404006958007812
(7, 33)
(65340, 7) (65340, 33)
30.7877140045166
(9, 33)
(65340, 9) (65340, 33)
32.013916015625


In [5]:
rec_data = cube_to_lut(rec_data)
print(rec_data.shape)
psnr = psnr_from_mse(np.power((luts-rec_data),2).mean(), 1).item()
print(psnr)
# torch.save(rec_data.reshape(n,-1), name + "_recluts_%.2f_%s.pth"%(psnr,str(condition)))
# torch.save(rec_data.reshape(n,-1), name + "_downluts_%s.pth"%(str(condition)))

torch.Size([20, 3, 33, 33, 33])
24.37532615661621


In [48]:
rec_data

tensor([[[[[ 1.6725e-02,  1.4597e-03, -2.8650e-02,  ..., -5.5312e-03,
             4.2640e-03,  7.8875e-03],
           [-6.1838e-03, -2.4121e-02, -4.7161e-02,  ..., -7.6882e-03,
             1.2340e-02,  1.2135e-02],
           [-1.6723e-02, -2.8695e-02, -5.2369e-02,  ..., -1.1640e-02,
             3.4253e-03,  2.7304e-04],
           ...,
           [ 6.5134e-03, -1.8812e-03,  4.6668e-03,  ..., -3.0255e-02,
            -1.0835e-02, -1.8832e-02],
           [ 8.3277e-03,  1.5449e-05,  8.1699e-03,  ..., -3.3184e-02,
            -1.1743e-02, -2.0236e-02],
           [ 9.2823e-03,  6.9114e-04,  1.5886e-02,  ..., -4.9048e-02,
            -1.5265e-02, -2.1817e-02]],

          [[ 2.6206e-02,  8.0642e-03, -2.3891e-02,  ...,  5.0636e-03,
            -4.0069e-03,  5.1480e-03],
           [ 4.9690e-03, -8.8034e-03, -2.2544e-02,  ..., -1.5370e-02,
            -8.1497e-03,  5.0830e-03],
           [-1.1724e-02, -2.3844e-02, -3.9149e-02,  ..., -1.0461e-02,
            -4.9192e-03, -1.0934e-04],
 

In [5]:
rec_data.max()

tensor(0.4486, dtype=torch.float64)

In [6]:
luts.min

<function Tensor.min>